# Simple SEIR Model for COVID-19 death toll prediction
Model design inspired by Henri Froese's [article](https://towardsdatascience.com/infectious-disease-modelling-beyond-the-basic-sir-model-216369c584c4)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint

## Design Ordinary differential equations (ODEs) for SEIR

In [19]:
def beta(t, R_0_start, R_0_end, t_lock, gamma, k=0.5):
    '''
    Expected amount of people an infected person infects per day
    Inputs:
        -t: point in time
        -R_0_start : R_0 at the start of epidemic
        -R_0_end: R_0 at the end of epidemic
        -t_lock: is the t-value of the inflection point (i.e.this could be thought of as the main “lockdown” date)
        -gamma:rate of recovery (1/Days the infection lasts)
        -k: how quickly R_0 declines
    
    Note: R_o(the total number of people an infected person infects (R₀ = β / γ))
    '''
    log_r_0 = (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+t_lock))) + R_0_end
    return log_r_0 * gamma

def odes_seir(pop, t, N, R_0_start, R_0_end, t_lock, gamma, delta, alpha, rho):
    '''
    Compute the ODE for Susceptible, Exposed, Infected, Recovered and Death with respect time(t)
    
    Inputs:
        -pop (list): includes susceptible, exposed, infected, and death populations
        -t: point in time (interger)
        -N: total population(fixed)
        -R_0_start : R_0 at the start of epidemic
        -R_0_end: R_0 at the end of epidemic
        -t_lock: is the t-value of the inflection point (i.e.this could be thought of as the main “lockdown” date)
        -gamma:rate of recovery (1/Days the infection lasts)
        -delta: length of incubation period
        -alpha: fatality rate
        -rho: rate at which people die (= 1/days from infected until death)
    
    '''
    S, E, I, R, D = pop
    dSusceptible_dt = -beta(t, R_0_start, R_0_end, t_lock, gamma)*S*I/N
    dExposed_dt = beta(t, R_0_start, R_0_end, t_lock, gamma)*S*I/N - delta*E
    dInfected_dt = delta * E- (1 - alpha) * gamma *I- alpha * rho *I
    dRecovered_dt = (1 - alpha) * gamma *I
    dDeath_dt = alpha * rho *I
    
    return dSusceptible_dt, dExposed_dt, dInfected_dt, dRecovered_dt, dDeath_dt

In [22]:
N = 1_000_000
D = 15 # infections lasts 15 days
gamma = 1.0 / D
delta = 1.0 / 2.0  # incubation period of five days
alpha = 0.1  # 10% death rate
R_0_start = 15
R_0_end =1
t_lock=40
rho = 1/7  #7 days from infection until death
S0, E0, I0, R0, D0 = N-1, 1, 0, 0, 0  # initial conditions: one exposed
t = np.linspace(0, 100, 100) # Grid of time points (in days)
y0 = S0, E0, I0, R0, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(odes_seir, y0, t, args=(N, R_0_start, R_0_end, t_lock, gamma, delta, alpha, rho))
S, E, I, R, D = ret.T

In [23]:
S

array([999999.        , 999998.77971166, 999998.1718349 , 999997.12048725,
       999995.42915792, 999992.74939282, 999988.51785357, 999981.84102969,
       999971.3077067 , 999954.69115705, 999928.47873354, 999887.12981437,
       999821.90579644, 999719.02608549, 999556.76344821, 999300.87265823,
       998897.40596338, 998261.44616124, 997259.49868544, 995682.12562874,
       993201.78955891, 989308.84173374, 983216.60804436, 973726.24492582,
       959047.89987962, 936596.87000584, 902841.04109487, 853393.41444866,
       783716.05936198, 690887.00609643, 576455.62006538, 449008.7159476 ,
       323491.2492027 , 215578.35802485, 134505.09792797,  80414.72672078,
        47476.59852155,  28576.51445955,  18047.569818  ,  12216.77919065,
         8954.91700588,   7093.40123293,   6002.17145264,   5340.21928489,
         4920.7463004 ,   4640.27544483,   4441.05562949,   4290.68220092,
         4170.8539058 ,   4071.12182088,   3985.41157729,   3910.09305196,
         3842.91135744,  